In [1]:
import os
import shutil
import random
import yaml

def split_dataset_with_yaml(original_dataset_dir, output_dataset_dir, original_yaml_path,
                             train_ratio=0.8, val_ratio=0.15, test_ratio=0.05, seed=42):
    random.seed(seed)

    images_dir = os.path.join(original_dataset_dir, "images")
    labels_dir = os.path.join(original_dataset_dir, "labels")

    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    image_files.sort()
    random.shuffle(image_files)

    n_total = len(image_files)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)

    train_files = image_files[:n_train]
    val_files = image_files[n_train:n_train + n_val]
    test_files = image_files[n_train + n_val:]

    def copy_files(subset_name, files):
        subset_img_dir = os.path.join(output_dataset_dir, subset_name, "images")
        subset_lbl_dir = os.path.join(output_dataset_dir, subset_name, "labels")
        os.makedirs(subset_img_dir, exist_ok=True)
        os.makedirs(subset_lbl_dir, exist_ok=True)

        for fname in files:
            shutil.copy(os.path.join(images_dir, fname), os.path.join(subset_img_dir, fname))
            label_name = os.path.splitext(fname)[0] + ".txt"
            label_path = os.path.join(labels_dir, label_name)
            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(subset_lbl_dir, label_name))

    copy_files("train", train_files)
    copy_files("val", val_files)
    copy_files("test", test_files)

    # 원래 data.yaml 읽기
    with open(original_yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)

    class_dict = {
    "concrete pipe": "materials",
    "concrete structure": "structure",
    "man-made structure": "structure",
    "equipments": "equipment",
    "equipment": "equipment",
    "crane": "equipment",
    "materials": "materials",
    "materials package": "materials",
    "rocks": "materials",
    "steel package": "materials",
    "textile": "others"
}
    # 새 경로로 수정
    new_yaml = {
        'train': os.path.join(output_dataset_dir, 'train', 'images'),
        'val': os.path.join(output_dataset_dir, 'val', 'images'),
        'test': os.path.join(output_dataset_dir, 'test', 'images'),
        'nc': data_yaml.get('nc'),
        'names': data_yaml.get('names'),
        'class_dict': class_dict
    }

    new_yaml_path = os.path.join(output_dataset_dir, 'data.yaml')
    with open(new_yaml_path, 'w') as f:
        yaml.dump(new_yaml, f)

    print(f"✅ Split complete: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")
    print(f"📄 New data.yaml saved to: {new_yaml_path}")
    return new_yaml_path

# 사용 예시
base_dir = "/workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair"
original_dataset = os.path.join(base_dir, "raw/train")
original_yaml = os.path.join(base_dir, "raw/data.yaml")
output_dataset = os.path.join(base_dir, "split")


new_yaml_path = split_dataset_with_yaml(original_dataset, output_dataset, original_yaml)


✅ Split complete: 200 train, 37 val, 14 test
📄 New data.yaml saved to: /workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/data.yaml


In [2]:
# crop image generation for sliding window
import os
import cv2
import yaml
import shutil
import numpy as np
from tqdm import tqdm

def sliding_window(image, step_size, window_size):
    """Sliding window generator with adjusted step size for edges."""
    h, w, _ = image.shape
    window_h, window_w = window_size

    for y in range(0, h - window_h + 1, step_size):
        for x in range(0, w - window_w + 1, step_size):
            yield x, y, image[y:y + window_h, x:x + window_w]

    # Handle the last window that may not fit within the current step size
    if (h - window_h) % step_size != 0:  # If there's space left in the vertical direction        
        y = h - window_h
        for x in range(0, w - window_w + 1, step_size):
            yield x, y, image[y:y + window_h, x:x + window_w]

    if (w - window_w) % step_size != 0:  # If there's space left in the horizontal direction                
        x = w - window_w
        for y in range(0, h - window_h + 1, step_size):
            yield x, y, image[y:y + window_h, x:x + window_w]

    # Handle the very last corner if step_size is larger than remaining space
    if (h - window_h) % step_size != 0 and (w - window_w) % step_size != 0:
        yield w - window_w, h - window_h, image[h - window_h:h, w - window_w:w]

def update_labels(label_path, x_offset, y_offset, window_size, img_shape, exclude_classes=[]):
    """Adjust bounding boxes for the cropped image, excluding specified classes."""
    updated_labels = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            cls, x_center, y_center, w, h = map(float, parts)
            if int(cls) in exclude_classes:
                continue  # Exclude specified classes
            x_center, y_center = x_center * img_shape[1], y_center * img_shape[0]
            w, h = w * img_shape[1], h * img_shape[0]
            
            x1, y1, x2, y2 = x_center - w / 2, y_center - h / 2, x_center + w / 2, y_center + h / 2
            
            x1 = max(x1, x_offset)
            y1 = max(y1, y_offset)
            x2 = min(x2, x_offset + window_size[0])
            y2 = min(y2, y_offset + window_size[1])
            
            if x2 > x1 and y2 > y1:
                x_new = ((x1 + x2) / 2 - x_offset) / window_size[0]
                y_new = ((y1 + y2) / 2 - y_offset) / window_size[1]
                w_new = (x2 - x1) / window_size[0]
                h_new = (y2 - y1) / window_size[1]
                updated_labels.append(f"{int(cls)} {x_new} {y_new} {w_new} {h_new}\n")
    return updated_labels

def preprocess_dataset(dataset_yaml, output_dir, reference_height = 2160, reference_altitude = 90, altitude = 90, step_ratio=0.5, exclude_classes=[]):
    """Process dataset images using sliding window and save cropped images/labels."""

    with open(dataset_yaml, 'r') as f:
        data_config = yaml.safe_load(f)

    # data.yaml이 위치한 디렉토리
    yaml_dir = os.path.dirname(os.path.abspath(dataset_yaml))    
    
    for split in ['train', 'val', 'test']:        
        rel_img_path = data_config[split]

        if not os.path.isabs(rel_img_path):
            # 상대경로인 경우
            img_dir = os.path.abspath(os.path.join(yaml_dir, rel_img_path))
        else:
            # 절대경로인 경우 그대로 사용
            img_dir = rel_img_path

        print(img_dir)

        output_img_dir = os.path.join(output_dir, split, 'images')
        output_label_dir = os.path.join(output_dir, split, 'labels')
        os.makedirs(output_img_dir, exist_ok=True)
        os.makedirs(output_label_dir, exist_ok=True)
        
        
        for img_name in tqdm(os.listdir(img_dir), desc=f"Processing {split} set"):
            
            img_path = os.path.join(img_dir, img_name)
            parent_dir = os.path.dirname(img_dir)
            dir_name = os.path.basename(img_dir)            

            label_dir = os.path.join(parent_dir, dir_name.replace('images', 'labels'))
            label_path = os.path.join(label_dir, img_name.replace('.jpg', '.txt'))
            img = cv2.imread(img_path)
            
            if img is None:
                continue
                        
            img_height = img.shape[0]
            window_size = int(640 * img_height / reference_height * reference_altitude / altitude)
            window_shape = (window_size, window_size)
            step_size = int(window_size * step_ratio)
            
            for idx, (x, y, window) in enumerate(sliding_window(img, step_size, window_shape)):
                crop_name = f"{os.path.splitext(img_name)[0]}_{idx}.jpg"
                crop_label_name = crop_name.replace('.jpg', '.txt')                                
                cv2.imwrite(os.path.join(output_img_dir, crop_name), window)

                updated_labels = update_labels(label_path, x, y, window_shape, img.shape, exclude_classes)
                if updated_labels:
                    with open(os.path.join(output_label_dir, crop_label_name), 'w') as f:
                        f.writelines(updated_labels)
    
    new_yaml = {
        'train': os.path.join(output_dir, 'train', 'images'),
        'val': os.path.join(output_dir, 'val', 'images'),
        'test': os.path.join(output_dir, 'test', 'images'),
        'nc': data_config['nc'],
        'names': data_config['names']
    }
    with open(os.path.join(output_dir, 'data.yaml'), 'w') as f:
        yaml.dump(new_yaml, f)
    
    return os.path.join(output_dir, 'data.yaml')


# 원본 데이터셋 경로 및 새로운 데이터셋 경로
data_yaml = "/workspace/Data/Sintanjin/240607_240726_Sintanjin/objectdetection_dataset/earthwork_80m_nerf_cd_pair_gps/split/data.yaml"
output_dir = "/workspace/Data/Sintanjin/240607_240726_Sintanjin/objectdetection_dataset/earthwork_80m_nerf_cd_pair_gps/crop"

# 제외할 클래스 지정 (예: 'human' 클래스가 0번일 경우)
exclude_classes = []

altitude = 80


# 데이터 전처리 실행
if os.path.exists(output_dir) and os.path.isdir(output_dir):
    print(f"[Info] '{output_dir}' already exists. Skipping preprocessing.")
    new_yaml_path = os.path.join(output_dir, 'data.yaml')  # 기존 yaml 경로 지정

else:
    new_yaml_path = preprocess_dataset(
        data_yaml,
        output_dir,
        altitude = altitude,
        exclude_classes=exclude_classes
    )

/workspace/Data/Sintanjin/240607_240726_Sintanjin/objectdetection_dataset/earthwork_80m_nerf_cd_pair_gps/split/train/images


Processing train set: 100%|██████████| 68/68 [00:14<00:00,  4.68it/s]


/workspace/Data/Sintanjin/240607_240726_Sintanjin/objectdetection_dataset/earthwork_80m_nerf_cd_pair_gps/split/val/images


Processing val set: 100%|██████████| 12/12 [00:02<00:00,  4.86it/s]


/workspace/Data/Sintanjin/240607_240726_Sintanjin/objectdetection_dataset/earthwork_80m_nerf_cd_pair_gps/split/test/images


Processing test set: 100%|██████████| 6/6 [00:01<00:00,  4.31it/s]


In [6]:
from ultralytics import YOLO
# YOLO 모델 학습
yaml_path = "/workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/data.yaml"
model = YOLO("/workspace/Laboratory/04.model/yolo/yolo11x.pt") # yolo11x-based
project_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO"
is_resume = False
name = "full_1280"
 
results = model.train(data=yaml_path, epochs=100, imgsz=1280, device=[0, 1], pretrained=True, project=project_path, name = name, resume = is_resume)

New https://pypi.org/project/ultralytics/8.3.155 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.38 🚀 Python-3.8.20 torch-2.1.0+cu118 CUDA:0 (NVIDIA RTX A6000, 48570MiB)
                                                      CUDA:1 (NVIDIA RTX A6000, 48561MiB)
engine/trainer: task=detect, mode=train, model=/workspace/Laboratory/04.model/yolo/yolo11x.pt, data=/workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO, name=full_1280, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, 

train: Scanning /workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/train/labels... 213 images, 22 backgrounds, 0 corrupt: 100%|██████████| 213/213 [00:00<00:00, 949.58it/s]


train: New cache created: /workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/train/labels.cache


val: Scanning /workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/val/labels... 35 images, 3 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<00:00, 1748.19it/s]


val: New cache created: /workspace/Data/240617_NajooGwangjoo/object_detection_dataset/earthwork_90m_nerf_cd_pair/split/val/labels.cache
Plotting labels to /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO/full_1280/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO/full_1280
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         35        433      0.959      0.946      0.974       0.76

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      34.8G     0.5468     0.2715     0.8414         74       1280: 100%|██████████| 14/14 [00:09<00:00,  1.44it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.61it/s]


                   all         35        433      0.961      0.947      0.975      0.763

100 epochs completed in 0.342 hours.
Optimizer stripped from /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO/full_1280/weights/last.pt, 114.5MB
Optimizer stripped from /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO/full_1280/weights/best.pt, 114.5MB

Validating /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO/full_1280/weights/best.pt...
Ultralytics 8.3.38 🚀 Python-3.8.20 torch-2.1.0+cu118 CUDA:0 (NVIDIA RTX A6000, 48570MiB)
                                                      CUDA:1 (NVIDIA RTX A6000, 48561MiB)
YOLO11x summary (fused): 464 layers, 56,840,884 parameters, 0 gradients, 194.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.28it/s]


                   all         35        433      0.941       0.96       0.97      0.766
                   car         16         32      0.987          1      0.995      0.892
         concrete pipe         12         25      0.866       0.92      0.939      0.828
                 crane          4          8      0.796      0.978      0.971      0.626
             equipment         20         60      0.972          1      0.995      0.848
              facility          9         19      0.976          1      0.995      0.877
                 human          7         20      0.928        0.8      0.834      0.405
    man-made structure         30         65          1      0.962      0.994      0.668
             materials         23        160      0.984      0.963      0.985      0.761
                 rocks         19         36      0.946       0.98      0.994      0.844
               textile          5          8       0.95          1      0.995      0.911
Speed: 0.3ms preproce

In [11]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

class_dict = {
    "concrete pipe": "materials",
    "concrete structure": "structure",
    "equipment": "equipment",
    "materials": "materials",
    "materials package": "materials",
    "rocks": "materials",
    "steel package": "materials",
    "textile": "materials"
}

def sliding_window(image, step_size, window_size):
    """Sliding window generator with adjusted step size for edges."""
    h, w, _ = image.shape
    window_h, window_w = window_size

    for y in range(0, h - window_h + 1, step_size):
        for x in range(0, w - window_w + 1, step_size):
            yield x, y, image[y:y + window_h, x:x + window_w]

    # Handle the last window that may not fit within the current step size
    if (h - window_h) % step_size != 0:  # If there's space left in the vertical direction        
        y = h - window_h
        for x in range(0, w - window_w + 1, step_size):
            yield x, y, image[y:y + window_h, x:x + window_w]

    if (w - window_w) % step_size != 0:  # If there's space left in the horizontal direction                
        x = w - window_w
        for y in range(0, h - window_h + 1, step_size):
            yield x, y, image[y:y + window_h, x:x + window_w]

    # Handle the very last corner if step_size is larger than remaining space
    if (h - window_h) % step_size != 0 and (w - window_w) % step_size != 0:
        yield w - window_w, h - window_h, image[h - window_h:h, w - window_w:w]

def perform_detection_on_sliding_window(model, class_names, image, step_size, window_shape):
    """Run object detection on the window using YOLO model."""
    
    result_image = image.copy()  # This will hold the final image with bounding boxes
    all_boxes = defaultdict(list)  # Dictionary to hold boxes per class

    # Run the sliding window and process each crop
    for x, y, window in sliding_window(image, step_size, window_shape):
        # Perform detection on the current window
        results  = model(window)
                
        # Extract the detected bounding boxes and their labels
        for result in results[0].boxes.data:  # Results format: [x1_, y1_, x2_, y2_, confidence, class]
            x1_, y1_, x2_, y2_, confidence, class_id = result

            # Convert to original image coordinates
            x1 = int(x1_ + x)
            y1 = int(y1_ + y)
            w = int(x2_ + x) - x1
            h = int(y2_ + y) - y1
            
            # Store the box coordinates along with confidence and class_id
            all_boxes[class_id.item()].append([x1, y1, w, h, confidence.item(), class_id.item()])

    # After collecting all boxes, apply NMS per class
    for class_id, boxes in all_boxes.items():
        # Extract coordinates, scores, and apply NMS
        bboxes = [box[:4] for box in boxes]             
        scores = [box[4] for box in boxes]
        # nms_boxes = apply_nms(bboxes, scores)
        nms_boxes = bboxes

        # Draw the bounding boxes after NMS on the result image
        for idx, box in enumerate(nms_boxes):
            x1, y1, w, h = box

            color = get_class_color(int(class_id))  

                    
            cv2.rectangle(result_image, (x1, y1), (x1 + w, y1 + h), color, 2)

            # Get the class name and confidence
            label = class_names[int(class_id)]  # Convert class_id to integer index for class_names

            group_label = class_dict.get(label, label)

            confidence = round(scores[idx], 2)
            text = f"{group_label} {confidence}"

            # Draw the class label on the image
            cv2.putText(result_image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 3)

    return result_image
 
def perform_detection_on_full_image(model, class_names, image):
    """Run object detection on the window using YOLO model."""
    result_image = image.copy()  # This will hold the final image with bounding boxes
    all_boxes = defaultdict(list)  # Dictionary to hold boxes per class

    # Perform detection on the image
    results  = model(image)
            
    # Extract the detected bounding boxes and their labels
    for result in results[0].boxes.data:  # Results format: [x1_, y1_, x2_, y2_, confidence, class]
        x1_, y1_, x2_, y2_, confidence, class_id = result

        # Convert to original image coordinates
        x1 = int(x1_)
        y1 = int(y1_)
        w = int(x2_) - x1
        h = int(y2_) - y1
        
        # Store the box coordinates along with confidence and class_id
        all_boxes[class_id.item()].append([x1, y1, w, h, confidence.item(), class_id.item()])

    # After collecting all boxes, apply NMS per class
    for class_id, boxes in all_boxes.items():
        # Extract coordinates, scores, and apply NMS
        bboxes = [box[:4] for box in boxes]             
        scores = [box[4] for box in boxes]
        # nms_boxes = apply_nms(bboxes, scores)
        nms_boxes = bboxes

        # Draw the bounding boxes after NMS on the result image
        for idx, box in enumerate(nms_boxes):
            x1, y1, w, h = box

            color = get_class_color(int(class_id))  

                    
            cv2.rectangle(result_image, (x1, y1), (x1 + w, y1 + h), color, 2)

            # Get the class name and confidence
            label = class_names[int(class_id)]  # Convert class_id to integer index for class_names

            group_label = class_dict.get(label, label)

            confidence = round(scores[idx], 2)
            text = f"{group_label} {confidence}"

            # Draw the class label on the image
            cv2.putText(result_image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 3)

    return result_image

def apply_nms(boxes, scores, threshold=0.7):
    """Apply Non-Maximum Suppression to the boxes."""
    
    indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.5, nms_threshold=threshold)
    
    # Extract surviving boxes (return in [x, y, w, h] format again)
    if len(indices) > 0:
        indices = indices.flatten()
        selected = [boxes[i] for i in indices]
        return selected
    else:
        return []

def get_class_color(class_id):
    """Return a unique color for each class."""
    colors = [
        (255, 0, 0),    # cars - Blue
        (0, 255, 0),    # clutter - Green
        (0, 0, 255),    # equipments - Red
        (255, 255, 0),  # facility - Cyan
        (255, 0, 255),  # human - Magenta
        (0, 255, 255),  # man-made structure - Yellow
        (128, 0, 128),  # materials - Purple
        (0, 128, 128),  # puddle - Teal
        (128, 128, 0),  # rocks - Olive
        (64, 64, 64)    # textile - Gray
    ]
    return colors[class_id % len(colors)]  # Ensure it doesn't go out of range

def process_full_image(image, model, class_names, sliding_window = False, reference_height = 2160, reference_altitude = 90, altitude = 90, step_ratio = 0.5):
    """Process the full image using sliding window and perform object detection."""
    height, width, _ = image.shape

    if sliding_window:
        window_size = int(1280 * height / reference_height * reference_altitude / altitude)
        window_shape = (window_size, window_size)
        step_size = int(window_size * step_ratio)

        result_image = perform_detection_on_sliding_window(model, class_names, image, step_size, window_shape)

    else:
        result_image = perform_detection_on_full_image(model, class_names, image)

    return result_image
    
    

# Load pre-trained YOLO model
model = YOLO("/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/YOLO/full/weights/best.pt")

# Define class names (you should update this list with the correct class names used in your model)
class_names = [
    'cars',
    'clutter',
    'concrete structure',
    'equipments',
    'facility',
    'human',
    'man-made structure',
    'materials',
    'puddle',
    'rocks',
    'textile'
]  # Replace with actual class names

# Load the 3840x2160 image
image_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Naju_90m_ChangeDetection_workspace/trial5/CD_PAIR/nerf_cd_pair_rectified/initial_paired_frame_00097.jpg"
image = cv2.imread(image_path)

# Perform detection on the full image
output_image = process_full_image(image, model, class_names, sliding_window=False)

# Save or display the output image with bounding boxes
cv2.imwrite("/workspace/Data/ObjectDetection/25-1_SCD-CON_Paper/vis/test.jpg", output_image)



0: 384x640 1 clutter, 7 equipmentss, 5 man-made structures, 17 materialss, 1 puddle, 3 rockss, 68.2ms
Speed: 25.8ms preprocess, 68.2ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)


True

In [18]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen

TypeError: 'NoneType' object is not iterable